In [ ]:
import numpy as np
from transformers import PreTrainedTokenizerFast
import torch
import pandas as pd
#from Functions_generation import generate_a_song_structure, sample_with_temp_topk, Subword_LSTM

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', '..')))
from Functions_generation import generate_a_song_structure, sample_with_temp_topk, Subword_RNN

In [ ]:
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="../../../Corpus/Encoding_RNN_LSTM/Subword/rap_tokenizer.json",
    pad_token = "<PAD>")

In [ ]:
matrix = pd.read_csv("../../../Markov/transition_matrix.csv")

states = np.array(matrix.iloc[6])
prob_transi = np.array(matrix.iloc[0:6])

In [ ]:
vocab_size = tokenizer.vocab_size
model = Subword_RNN(vocab_size)   

In [ ]:
ckpt = torch.load("../Models/RNN_model.pt", map_location="cpu")

new_state_dict = {}
for key, value in ckpt["model_state_dict"].items():
    new_key = key.replace("_orig_mod.", "")           
    new_state_dict[new_key] = value

ckpt["model_state_dict"] = new_state_dict

model.load_state_dict(ckpt["model_state_dict"])

I will give the same structure for RNN and LSTM to compare

In [ ]:
struct = generate_a_song_structure(prob_transi.astype(float),states)

#struct = ['<BEGINNING>', '<COUPLET>', '<REFRAIN>', '<COUPLET>', '<REFRAIN>', '<END>']
struct = ['α', 'γ', 'ε', 'γ', 'ε', 'θ']

encoded = torch.tensor([mapping[c] for c in struct[1]], dtype=torch.long).unsqueeze(0)

Forbidden to generate

In [ ]:
forbidden_generation = [i for i in range(64,72)]
[int2char[i] for i in forbidden_generation]

In [ ]:
decod_structure = {"β" : "<INTRO>",
                   "γ" : "<COUPLET>",
                   "ε" : "<REFRAIN>",
                   "ζ" : "<PONT>",
                   "η" : "<OUTRO>",
                   "θ" : "<END>"}

In [ ]:
context_gen = "voila pourquoi"
context_encode = [mapping[i] for i in context_gen]
context_size = len(context_encode)

hid = model.init_hidden(batch_size=1)

for i in range(len(struct) - 2):

    print()
    print(decod_structure[struct[i+1]])

    if i == 0 : 
        context_encode.insert(0,mapping[struct[i+1]])
        context_encode.insert(1,mapping["\n"])
        encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)

        print("".join([int2char[i] for i in context_encode[1:]]), end="")

        out, hid = model(encoded, hid)
        next_input = encoded[:, -1].unsqueeze(0)  # shape (1, 1)

    else : 
        #next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).unsqueeze(0)
        next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).view(1,1)

    last_input = 0
    first_input = True

    while last_input != 12:
        with torch.no_grad():
            out, hid = model(next_input, hid)   # out shape: (1, 1, vocab_size)

            out[0, -1][forbidden_generation] = float("-inf")

        # 4. On échantillonne un token
            next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)
            last_input = next_token.item()

        # 5. Conditions d’impression
            if first_input and last_input == 12:  # Si début et le modèle sort directement <END>
                last_input = 0
                continue

            elif first_input and last_input == 0:  # Si début et sortie = \n
                first_input = False
                forbidden_generation.extend([0])
                print(int2char[next_token.item()], end="")

            elif next_token.item() != 12:
                first_input = False
                print(int2char[next_token.item()], end="")

                if 0 in forbidden_generation:
                    forbidden_generation.pop(-1)

        # 6. Préparer la prochaine entrée
            next_input = next_token.view(1, 1)

print("\n", decod_structure[struct[-1]])


# If no restriction on the generation part 

In [ ]:
context_gen = "malice"
context_encode = [mapping[i] for i in context_gen]

hid = model.init_hidden(batch_size=1)

encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)
print("".join([int2char[i] for i in context_encode]), end="")

out, hid = model(encoded, hid)
next_input = encoded[:, -1].unsqueeze(0) 

last_input = 0
first_input = True

while last_input != 70:
    with torch.no_grad():
        out, hid = model(next_input, hid) 

        next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)

        if (last_input == 0) and (next_token != list(decod_structure.keys())):
            print(int2char[next_token.item()].upper(), end="")
        elif last_input != 70 : 
            print(int2char[next_token.item()], end="")

        last_input = next_token.item()
        next_input = next_token.view(1, 1)

# Teacher forcing < 1 toward the end of training

In [ ]:
ckpt = torch.load("..\Models\RNN_model_not_full_forced.pt", map_location="cpu")
new_state_dict = {}
for k, v in ckpt["model_state_dict"].items():
    new_key = k.replace("_orig_mod.", "")  # supprime le préfixe "origin."
    new_state_dict[new_key] = v

In [ ]:
#ckpt = torch.load("..\Models\LSTM_model_not_full_forced.pt", map_location="cpu")
#torch.compile(model)
model.load_state_dict(new_state_dict)

Forbidden to generate

In [ ]:
context_gen = "voila pourquoi"
context_encode = [mapping[i] for i in context_gen]
context_size = len(context_encode)

hid = model.init_hidden(batch_size=1)

for i in range(len(struct) - 2):

    print()
    print(decod_structure[struct[i+1]])

    if i == 0 : 
        context_encode.insert(0,mapping[struct[i+1]])
        context_encode.insert(1,mapping["\n"])
        encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)

        print("".join([int2char[i] for i in context_encode[1:]]), end="")

        out, hid = model(encoded, hid)
        next_input = encoded[:, -1].unsqueeze(0)  # shape (1, 1)

    else : 
        #next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).unsqueeze(0)
        next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).view(1,1)

    last_input = 0
    first_input = True

    while last_input != 12:
        with torch.no_grad():
            out, hid = model(next_input, hid)   # out shape: (1, 1, vocab_size)

            out[0, -1][forbidden_generation] = float("-inf")

        # 4. On échantillonne un token
            next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)
            last_input = next_token.item()

        # 5. Conditions d’impression
            if first_input and last_input == 12:  # Si début et le modèle sort directement <END>
                last_input = 0
                continue

            elif first_input and last_input == 0:  # Si début et sortie = \n
                first_input = False
                forbidden_generation.extend([0])
                print(int2char[next_token.item()], end="")

            elif next_token.item() != 12:
                first_input = False
                print(int2char[next_token.item()], end="")

                if 0 in forbidden_generation:
                    forbidden_generation.pop(-1)

        # 6. Préparer la prochaine entrée
            next_input = next_token.view(1, 1)

print("\n", decod_structure[struct[-1]])


# If no restriction on the generation part 

In [ ]:
context_gen = "malice"
context_encode = [mapping[i] for i in context_gen]

hid = model.init_hidden(batch_size=1)

encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)
print("".join([int2char[i] for i in context_encode]), end="")

out, hid = model(encoded, hid)
next_input = encoded[:, -1].unsqueeze(0) 

last_input = 0
first_input = True

while last_input != 70:
    with torch.no_grad():
        out, hid = model(next_input, hid) 

        next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)

        if (last_input == 0) and (next_token != list(decod_structure.keys())):
            print(int2char[next_token.item()].upper(), end="")
        elif last_input != 70 : 
            print(int2char[next_token.item()], end="")

        last_input = next_token.item()
        next_input = next_token.view(1, 1)

In [ ]:
matrix = pd.read_csv(r"..\Markov\transition_matrix.csv")

states = np.array(matrix.iloc[6])
prob_transi = np.array(matrix.iloc[0:6])

# Model loading

In [ ]:
vocab_size = len(int2char)
hidden_size = 250

model = CharRNN(vocab_size, hidden_size, num_layers=3)   

ckpt = torch.load("..\Models\RNN_model.pt", map_location="cpu")
model.load_state_dict(ckpt["model_state_dict"])

The char token representing the end of a section is at place 79

In [ ]:
struct = generate_a_song_structure(prob_transi.astype(float),states)

encoded = torch.tensor([mapping[c] for c in struct[1]], dtype=torch.long).unsqueeze(0)

In [ ]:
hid = model.init_hidden(batch_size=1)

for i in range(len(struct)-2) :

    print()
    print(struct[i+1])
    encoded = torch.tensor([mapping[c] for c in struct[i+1]], dtype=torch.long).unsqueeze(0)

    # prime the model with your encoded context
    out, hid = model(encoded, hid)

    # take the last token of the context
    next_input = encoded[:, -1].unsqueeze(0)  # shape (1, 1)

    last_input = 0
    first_input = True

    while last_input != 79 :
        with torch.no_grad():
            out, hid = model(next_input, hid)   # out shape: (1, 1, vocab_size)

    # sample from logits
        next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)
        last_input = next_token.item()

    # print char
        if first_input and last_input == 79 :
            last_input = 0
            continue

        if next_token.item() != 79 :
            first_input = False
            print(int2char[next_token.item()], end="")


    # prepare next input (still (1, 1))
        next_input = next_token.view(1, 1)

print("\n",struct[-1])